In [1]:
### Allow relative import - Only relevant for .ipynb files in /docs
import sys
import os
import random
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

sys.path.append(os.path.abspath(os.pardir))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sktime.datasets import load_UCR_UEA_dataset
from sklearn.model_selection import train_test_split

from ROCKAD import ROCKAD
from ROCKAD import NearestNeighborOCC

# Set random state for reproducibility
RANDOM_STATE = 42


#### Univariate Anomaly Detection


In [3]:
# Load univariate data
X, y = load_UCR_UEA_dataset("GunPoint", return_X_y=True)

# Split the data and produce anomalies
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# Create the normal dataset (Normal class: Class 1)  
#        the anomaly dataset (Anomaly class: Class 2)
X_normal_train = X_train[y_train == '1']

X_normal_test = X_test[y_test == '1']
X_anomaly_test = X_test[y_test == '2']
y_normal_test = y_test[y_test == '1']
y_anomaly_test = y_test[y_test == '2']

# Merge the test sets 
X_test = pd.DataFrame(np.concatenate((X_normal_test, X_anomaly_test), axis=0))
y_test = np.concatenate((y_normal_test, y_anomaly_test), axis=0)

In [4]:
# Initialize and fit ROCKAD
rockad = ROCKAD(random_state=RANDOM_STATE).fit(X_normal_train) 

# Predict anomaly scores
scores = rockad.predict_proba(X_test)

In [5]:
# Initialize and fit NearestNeigbor One Class Classifier
decision_func = NearestNeighborOCC().fit(scores)

# Predict anomalies
predictions = decision_func.predict(scores)

#### Multivariate Anomaly Detection

In [6]:
# Load multivariate data
X, y = load_UCR_UEA_dataset("BasicMotions", return_X_y=True)

# Split the data and produce anomalies
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# Create the normal dataset (Normal class: running, walking, badminton)  
#        the anomaly dataset (Anomaly class: standing)
X_normal_train = X_train[y_train != 'standing']

X_normal_test = X_test[y_test != 'standing']
X_anomaly_test = X_test[y_test == 'standing']
y_normal_test = y_test[y_test != 'standing']
y_anomaly_test = y_test[y_test == 'standing']

# # Merge the test sets 
X_test = pd.DataFrame(np.concatenate((X_normal_test, X_anomaly_test), axis=0))
y_test = np.concatenate((y_normal_test, y_anomaly_test), axis=0)

In [7]:
# Initialize and fit ROCKAD
rockad = ROCKAD(random_state=RANDOM_STATE).fit(X_normal_train) 

# Predict anomaly scores
scores = rockad.predict_proba(X_test)

In [8]:
# Initialize and fit NearestNeigbor One Class Classifier
decision_func = NearestNeighborOCC().fit(scores)

# Predict anomalies
predictions = decision_func.predict(scores)